In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [35]:
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import mse
# Load the model with custom objects
model = load_model('hurri_pred.h5', custom_objects={'mse': mse})

In [36]:
# Let's clean the data by handling the NaN values and ensuring all relevant columns are ready for model input.
# We'll fill NaN values with appropriate strategies (e.g., forward fill for continuity or filling with mean values for wind and pressure).
df = pd.read_csv('ModifiedHurriData.csv')
df = df.sort_values(axis=0,by=['name','year','month','day'])
# Dropping the 'Unnamed: 0', 'name', 'year', 'month', 'day', 'hour', 'status', and 'category' columns since they're not needed for the model
df_cleaned = df.drop(columns=['H1', 'status', 'category'])

# Forward fill the latitude and longitude to maintain continuity
df_cleaned[['lat', 'long']] = df_cleaned[['lat', 'long']].fillna(method='ffill')

# Fill NaN values in 'wind' and 'pressure' with the mean of each column
df_cleaned['wind'].fillna(df_cleaned['wind'].mean(), inplace=True)
df_cleaned['pressure'].fillna(df_cleaned['pressure'].mean(), inplace=True)

# Fill NaN values in 'tropicalstorm_force_diameter' and 'hurricane_force_diameter' with 0 (assuming no force if NaN)
df_cleaned['tropicalstorm_force_diameter'].fillna(0, inplace=True)
df_cleaned['hurricane_force_diameter'].fillna(0, inplace=True)

# Display the first few rows of the cleaned data
df_cleaned.head()


C:\Users\Kenne\AppData\Local\Temp\ipykernel_49904\2677572207.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Kenne\AppData\Local\Temp\ipykernel_49904\2677572207.py:12: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Kenne\AppData\Local\Temp\ipykernel_49904\2677572207.py:13: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior w

,name,year,month,day,hour,lat,long,wind,pressure,tropicalstorm_force_diameter,hurricane_force_diameter
4544,AL011993,1993,5,31,12,21.5,-84.0,25,1003,0.0,0.0
4545,AL011993,1993,5,31,18,22.3,-82.0,25,1002,0.0,0.0
4546,AL011993,1993,6,1,0,23.2,-80.3,25,1000,0.0,0.0
4547,AL011993,1993,6,1,6,24.5,-79.0,25,1000,0.0,0.0
4548,AL011993,1993,6,1,12,25.4,-77.5,30,999,0.0,0.0


In [37]:
# Assuming df_cleaned is your cleaned dataframe
unique_hurricanes = df_cleaned['name'].unique()
random_hurricane = np.random.choice(unique_hurricanes)

# Get data for the selected hurricane
hurricane_data = df_cleaned[df_cleaned['name'] == random_hurricane].sort_values(axis=0,by=['name','year','month','day'])

In [38]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df_cleaned is your cleaned dataframe
features = ['wind', 'pressure', 'lat', 'long']

# Create and fit the scaler
scaler = MinMaxScaler()
scaler.fit(df_cleaned[features])

# Now you can use this scaler for transforming and inverse transforming your data

MinMaxScaler()

In [39]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        sequences.append(seq)
    return np.array(sequences)

seq_length = 10  # Should be the same as used during training
features = ['wind', 'pressure', 'lat', 'long']

hurricane_scaled = scaler.transform(hurricane_data[features])
X = create_sequences(hurricane_scaled, seq_length)

In [40]:
# Make predictions
predictions = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step


In [41]:
lat_long_scaler = MinMaxScaler()
lat_long_scaler.fit(df_cleaned[['lat', 'long']])

MinMaxScaler()

In [42]:
# Prepare actual values
actual_positions = hurricane_scaled[seq_length:, 2:4]  # lat, long
# Inverse transform using the lat_long_scaler
predictions_original = lat_long_scaler.inverse_transform(predictions)
actual_positions_original = lat_long_scaler.inverse_transform(actual_positions)

In [43]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Assuming you've already done the predictions and inverse transformations

# Create a DataFrame for actual positions
actual_df = pd.DataFrame({
    'lat': actual_positions_original[:, 0],
    'lon': actual_positions_original[:, 1],
    'type': 'Actual',
    'time_step': range(len(actual_positions_original))
})

# Create a DataFrame for predicted positions
predicted_df = pd.DataFrame({
    'lat': predictions_original[:, 0],
    'lon': predictions_original[:, 1],
    'type': 'Predicted',
    'time_step': range(len(predictions_original))
})

# Combine the DataFrames
df = pd.concat([actual_df, predicted_df])

# Create the figure
fig = go.Figure()

# Add actual path
fig.add_trace(go.Scattermapbox(
    lat=actual_df['lat'],
    lon=actual_df['lon'],
    mode='lines+markers',
    marker=go.scattermapbox.Marker(
        size=8
    ),
    line=dict(width=2, color='blue'),
    name='Actual Path'
))

# Add predicted path
fig.add_trace(go.Scattermapbox(
    lat=predicted_df['lat'],
    lon=predicted_df['lon'],
    mode='lines+markers',
    marker=go.scattermapbox.Marker(
        size=8, 
        showscale=False
    ),
    line=dict(width=2, color='rgba(255,0,0,0.5)'),  # Red with 50% opacity
    name='Predicted Path'
))

# Add start point
fig.add_trace(go.Scattermapbox(
    lat=[actual_df['lat'].iloc[0]],
    lon=[actual_df['lon'].iloc[0]],
    mode='markers',
    marker=go.scattermapbox.Marker(size=12, color='green'),
    name='Start'
))

# Add end point
fig.add_trace(go.Scattermapbox(
    lat=[actual_df['lat'].iloc[-1]],
    lon=[actual_df['lon'].iloc[-1]],
    mode='markers',
    marker=go.scattermapbox.Marker(size=12, color='purple'),
    name='End'
))

# Update the layout
fig.update_layout(
    title=f'Actual vs Predicted Path for Hurricane {random_hurricane}',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=actual_df['lat'].mean(), lon=actual_df['lon'].mean()),
        zoom=5
    ),
    showlegend=True
)

# Show the plot
fig.show()

print(f"Hurricane: {random_hurricane}")
print(f"Start Position: Lat {actual_positions_original[0, 0]:.2f}, Lon {actual_positions_original[0, 1]:.2f}")
print(f"End Position: Lat {actual_positions_original[-1, 0]:.2f}, Lon {actual_positions_original[-1, 1]:.2f}")

Hurricane: Dolly-2002
Start Position: Lat 13.60, Lon -46.80
End Position: Lat 24.00, Lon -52.50


In [44]:
actual_df = pd.DataFrame({
    'lat': actual_positions_original[:, 0],
    'lon': actual_positions_original[:, 1],
    'type': 'Actual',
    'time_step': range(len(actual_positions_original))
})

# Create a DataFrame for predicted positions
predicted_df = pd.DataFrame({
    'lat': predictions_original[:, 0],
    'lon': predictions_original[:, 1],
    'type': 'Predicted',
    'time_step': range(len(predictions_original))
})

# Combine the DataFrames
df = pd.concat([actual_df, predicted_df])

# Create the figure
fig = go.Figure()

# Add actual path
fig.add_trace(go.Scattermapbox(
    lat=actual_df['lat'],
    lon=actual_df['lon'],
    mode='lines+markers',
    marker=go.scattermapbox.Marker(
        size=8
    ),
    line=dict(width=2, color='blue'),
    name='Actual Path'
))

# Add predicted path


# Add start point
fig.add_trace(go.Scattermapbox(
    lat=[actual_df['lat'].iloc[0]],
    lon=[actual_df['lon'].iloc[0]],
    mode='markers',
    marker=go.scattermapbox.Marker(size=12, color='green'),
    name='Start'
))

# Add end point
fig.add_trace(go.Scattermapbox(
    lat=[actual_df['lat'].iloc[-1]],
    lon=[actual_df['lon'].iloc[-1]],
    mode='markers',
    marker=go.scattermapbox.Marker(size=12, color='purple'),
    name='End'
))

# Update the layout
fig.update_layout(
    title=f'Actual vs Predicted Path for Hurricane {random_hurricane}',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=actual_df['lat'].mean(), lon=actual_df['lon'].mean()),
        zoom=5
    ),
    showlegend=True
)

# Show the plot
fig.show()

print(f"Hurricane: {random_hurricane}")
print(f"Start Position: Lat {actual_positions_original[0, 0]:.2f}, Lon {actual_positions_original[0, 1]:.2f}")
print(f"End Position: Lat {actual_positions_original[-1, 0]:.2f}, Lon {actual_positions_original[-1, 1]:.2f}")

Hurricane: Dolly-2002
Start Position: Lat 13.60, Lon -46.80
End Position: Lat 24.00, Lon -52.50
